In [ ]:
! pip install elasticsearch==7.7.1
! pip install datasets

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.7.1-linux-x86_64.tar.gz
! tar -xzvf elasticsearch-7.7.1-linux-x86_64.tar.gz

 import datasets
 from lfqa_utils import *
 from eli5_utils import *
 import json

In [ ]:
es_client = Elasticsearch("http://localhost:9200")

In [ ]:
wiki40b_snippets = datasets.load_from_disk("../wiki40b/train")
eli5 = datasets.load_from_disk("../eli5_dataset/")

In [ ]:
question = eli5['test_eli5'][np.random.randint(0, len(eli5['test_eli5']))]['title']
doc, res_list = query_es_index(question, es_client, index_name='wiki40b_snippets_100w')

df = pd.DataFrame({
    'Article': ['---'] + [res['article_title'] for res in res_list],
    'Sections': ['---'] + [res['section_title'] if res['section_title'].strip() != '' else res['article_title']
                 for res in res_list],
    'Text': ['--- ' + question] + [res['passage_text'] for res in res_list],
})

df

In [ ]:
# same code for the askh and eli5 subsets

eli5_train_docs = []
for example in eli5["train_asks"]:
    support_doc, res_list = query_es_index(example["title"], es_client, index_name="wiki40b_snippets_100w")
    eli5_train_docs += [(example["q_id"], support_doc, res_list)]

eli5_valid_docs = []
for example in eli5["validation_asks"]:
    support_doc, res_list = query_es_index(example["title"], es_client, index_name="wiki40b_snippets_100w")
    eli5_valid_docs += [(example["q_id"], support_doc, res_list)]

with open("train_asks.json", "w") as f:
    json.dump(eli5_train_docs, f)

with open("valid_asks.json", "w") as f:
    json.dump(eli5_valid_docs, f)